In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=0)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([613, 613]))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       270
           1       0.76      0.83      0.79        30

    accuracy                           0.96       300
   macro avg       0.87      0.90      0.88       300
weighted avg       0.96      0.96      0.96       300



In [9]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [12]:
mlflow.set_experiment("imbalanced classification")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model") 

2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/24 21:14:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/24 21:14:02 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/24 21:14:02 INFO mlflow.store.db.utils: Updating database tables
2026/01/24 21:14:02 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/24 21:14:02 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/24 21:14:02 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/24 21:14:02 INFO alembic.runtime

🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/2/runs/fa617c179023422dabac40538d3f267c
🧪 View experiment at: http://localhost:5000/#/experiments/2


2026/01/24 21:14:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/2/runs/e560900a94674a6496ad88f5f11e15c5
🧪 View experiment at: http://localhost:5000/#/experiments/2


2026/01/24 21:15:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/2/runs/fab3fb0c08e54ad6a22347711745d875
🧪 View experiment at: http://localhost:5000/#/experiments/2


2026/01/24 21:16:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/2/runs/f4f05bfa745945d8844ec3f9e7e07ead
🧪 View experiment at: http://localhost:5000/#/experiments/2
